In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
from scipy.stats import chi2_contingency

import requests
from bs4 import BeautifulSoup

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

In [3]:
stopWords = ['and', 'at', 'the', 'yet', 'so', 'because', 'on', 'of', 'to', 'as',' in', 'his', 'her', 'she', 'him',
            'them', 'they', 'it', 'hers', 'their', 'its', 'theirs', 'with', 'said', 'for', 'after', 'will', 'that',
            'about', 'who', 'by', 'all', 'where', 'over', 'year', 'years', 'continue', 'two', 'three', 'four', 'five',
            'six', 'seven', 'eight', 'nine', 'ten', 'other', 'into']

In [4]:
def trimWord(word):
    w = word
    
    if ((w[len(w)-1] == ",")or(w[len(w)-1] == ".")or(w[len(w)-1] == "?")
        or(w[len(w)-1] == "!")or(w[len(w)-1] == "’")or(w[len(w)-1] == ":")
        or(w[len(w)-1]=="\"")or(w[len(w)-1]=="-")or(w[len(w)-1]=="'")):
        w = w[:len(w)-1]
                        
    if((w[0]=="'")or(w[0]=="’")or(w[0]=="\"")or(w[0]=="-")):
        w = w[1:]
                        
    if ((w[len(w)-2:len(w)] == "’s")or(w[len(w)-2:len(w)] == "\'s")):
        w = w[:len(w)-2]
    
    return w

In [5]:
def scrapeArticlesCNN(w, rBank, dBank):
    dataFile = open("cnn.txt", "a")
    
    www = w
    www = requests.get(www,headers = headers)

    if (www.status_code != 200):
        return "Error"

    soup = BeautifulSoup(www.text, 'html.parser')    
    
    headlineBlocks = soup.find_all('span', class_ = 'sitemap-link')
    headlineBlocks = headlineBlocks[1:]
    #for i in headlineBlocks:
    #    print(i)
    x=0
    for currentHeadline in headlineBlocks:
        x+=1
        articleWWW = str(currentHeadline.find('a')).split("\"")[1]
        #print(articleWWW)
        articleWWW = requests.get(articleWWW,headers = headers)
        
        if (www.status_code != 200):
            return "Error"
        
        soup = BeautifulSoup(articleWWW.text, 'html.parser')
        
        
        #-------------------------------------------------------------------------------------------------------------
        headline = soup.find('h1', class_="headline__text inline-placeholder")
        #print(headline.text.split())
        
        r = False
        d = False
        try:
            headline_words = headline.text.split()
        except:
            continue
            
            
        for word in headline_words:
            try:
                word = trimWord(word)
                word = word.lower()
            except:
                continue
                
            if(word in stopWords):
                continue
            
            if (not r):
                r = (word in rBank)
            if (not d):
                d = (word in dBank)
            
            
        if (not ((r or d) & (not (r&d)))):
            continue
        #------------------------------------------------------------------------------------------------------------- 
        paragraph = soup.find_all('p', class_ = 'paragraph inline-placeholder')
        
        try:
            if (r):
                dataFile.write(paragraph[0].text + paragraph[1].text + ' r     \n')
            if (d):
                dataFile.write(paragraph[0].text + paragraph[1].text + ' d     \n')
        except:
            continue
        

    dataFile.close()
    
    return "done"

In [7]:
www = "https://www.multistate.us/resources/2023-governors-and-legislatures"

www = requests.get(www,headers = headers)
if (www.status_code != 200):
    print("Error")

soup = BeautifulSoup(www.text, 'html.parser')

politicians = []
parties = []

govBlocks = soup.find_all("td", class_ = "gov left")

for govBlock in govBlocks:
    
    vect = govBlock.text.split()
    governor = vect[1]
    party = vect[len(vect)-1][1]
    
    if(party=="D"):
        party = "Democratic"
    if(party=="R"):
        party = "Republican"
            
    politicians.append(governor.lower())
    parties.append(party)

www = "https://ballotpedia.org/List_of_current_members_of_the_U.S._Congress"

www = requests.get(www,headers = headers)
if (www.status_code != 200):
    print("Error")

soup = BeautifulSoup(www.text, 'html.parser')
        
people = soup.find_all('tr')

for i in people:
    try:
        name = i.find('td', style="padding-left:10px;text-align:center;").text
        party = ""
        try:
            party = i.find("td", class_ = "partytd Democratic").text
            party = "Democratic"
        except:
            party = i.find("td", class_ = "partytd Republican").text
            party = "Republican"
        
        names = name.split()
        last_name = names[len(names)-1]
        politicians.append(last_name.lower())
        parties.append(party)
        
    except:
        continue
        


In [8]:
additionalRepublicans = ['trump', 'g.o.p.', 'gop', 'republicans', 'republican', 'conservatives', 'pence']
additionalDemocrats = ['pelosi', 'biden', 'democrat', 'democrats', 'liberals', 'kamala', 'obama', 'bernie', 'clinton']

In [9]:
people = ""
pdf = pd.DataFrame()
pdf['Politician'] = politicians
pdf['Party'] = parties

for a in additionalRepublicans:
    pdf.loc[len(pdf['Party'].values)] = a.lower()
    pdf['Party'].loc[len(pdf['Party'].values)-1] = 'Republican'

for b in additionalDemocrats:
    pdf.loc[len(pdf['Party'].values)] = b.lower()
    pdf['Party'].loc[len(pdf['Party'].values)-1] = 'Democratic'

In [10]:
#remove some politicians with generic names
removePoliticians = ['justice', 'cooper', 'cox', 'murphy', 'green', 'greene', 'little', 'moore', 'reeves', 'shapiro', 
                     'lee', 'scott', 'jr.', 'miller', 'rodgers', 'good', 'fitzgerald', 'smith', 'tiffany', 'carson', 
                    'stevens', 'james', 'harris', 'davis', 'carter', 'clark', 'jackson', 'collins', 'bush', 'thompson'
                    'gordon', 'young', 'paul', 'peters', 'brown', 'murray', 'iii', 'johnson' 'carl' 'rogers', 'strong',
                    'crane', 'hill', 'kim', 'steel', 'crow', 'courtney', 'bean', 'frost', 'posey', 'wilson', 'williams',
                    'allen', 'simpson', 'flood', 'ryan', 'ross', 'adams', 'edwards', 'jordan', 'lucas', 'cole', 'rose',
                    'fallon', 'roy', 'drew', 'clarke', 'wild', 'gordon', 'kennedy', 'schmitt', 'booker', 'vance', 
                     'mullin', 'reed', 'graham', 'evans', 'dean', 'duncan', 'case', 'hunt', 'golden', 'carey', 'joyce',
                    'johnson','graves', 'davidson', 'franklin', 'cassidy', 'garcia', 'mills', 'thompson', 'curtis',
                    'stewart', 'guest', 'cloud', 'banks', 'self', 'waters', 'buck', 'game', 'foster', 'rounds', 'carl',
                    'kiley', 'mast', 'rogers', 'owens', 'fry', 'norman', 'crenshaw']

for r in removePoliticians:
    try:
        pdf = pdf.drop(pdf.index[pdf['Politician'] == r.lower()].tolist())
        
    except:
        continue

In [11]:
republicans = list(pdf[pdf['Party'] == 'Republican']['Politician'])
democrats = list(pdf[pdf['Party'] == 'Democratic']['Politician'])
#print(democrats)

In [12]:
#print(republicans)

['ivey', 'dunleavy', 'huckabee', 'desantis', 'kemp', 'holcomb', 'reynolds', 'parson', 'gianforte', 'pillen', 'lombardo', 'sununu', 'burgum', 'dewine', 'stitt', 'mcmaster', 'noem', 'abbot', 'youngkin', 'britt', 'tuberville', 'murkowski', 'sullivan', 'boozman', 'cotton', 'rubio', 'crapo', 'risch', 'braun', 'ernst', 'grassley', 'marshall', 'moran', 'mcconnell', 'hyde-smith', 'wicker', 'hawley', 'daines', 'fischer', 'ricketts', 'budd', 'tillis', 'cramer', 'hoeven', 'lankford', 'thune', 'blackburn', 'hagerty', 'cornyn', 'cruz', 'romney', 'capito', 'barrasso', 'lummis', 'aderholt', 'palmer', 'radewagen', 'schweikert', 'biggs', 'ciscomani', 'lesko', 'gosar', 'crawford', 'womack', 'westerman', 'lamalfa', 'mcclintock', 'duarte', 'mccarthy', 'valadao', 'obernolte', 'calvert', 'issa', 'boebert', 'lamborn', 'gaetz', 'dunn', 'cammack', 'rutherford', 'waltz', 'webster', 'bilirakis', 'luna', 'buchanan', 'steube', 'donalds', 'diaz-balart', 'salazar', 'gimenez', 'ferguson', 'mccormick', 'clyde', 'loude

In [13]:
open('cnn.txt', 'w').close()
for month in range(1,3):
    print(scrapeArticlesCNN("https://www.cnn.com/politics/article/sitemap-2023-" + str(month) + ".html", republicans, democrats))